In [66]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
warnings.filterwarnings(action='ignore')

%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

plt.rcParams["font.family"] = 'NanumSquareRoundOTF'
plt.rcParams["font.size"] = 12

In [67]:
df_test = pd.read_csv('../Data/test_PCA_jmk_lnd.csv')
df_train = pd.read_csv('../Data/train_PCA_jmk_lnd.csv')
df_valid = pd.read_csv('../Data/valid_PCA_jmk_lnd.csv')
# 원본
df_te = pd.read_csv('../Data/PJT002_test.csv')
df_tr = pd.read_csv('../Data/PJT002_train.csv')
df_va = pd.read_csv('../Data/PJT002_validation.csv')

In [68]:
# 데이터 리스트
df_l = [df_train, df_test, df_valid]

In [69]:
# 자주 쓸 컬럼들
w = ['id', 'bldng_cnt', 'bldng_ar', 'ttl_ar', 'lnd_ar', 'ttl_grnd_flr', 'ttl_dwn_flr','lnd_us_sttn_nm(clean)']

In [70]:
# u = ['id', 'bldng_cnt', 'bldng_ar', 'ttl_ar', 'lnd_ar', 'ttl_grnd_flr', 'ttl_dwn_flr','lnd_us_sttn_nm']
# df_tr[df_tr['lnd_us_sttn_nm'] == '유해.혐오시설'][u]

## 지하 층 결측치

In [71]:
# 지하 층 컬럼 복원하기

# 지하 층 수 notull은 평균 0.13... 이므로 0으로 채움
a = df_te[['id','ttl_dwn_flr']].fillna(0)
df_test = df_test.merge(a, on='id')

a = df_tr[['id','ttl_dwn_flr']].fillna(0)
df_train = df_train.merge(a, on='id')

a = df_va[['id','ttl_dwn_flr']].fillna(0)
df_valid = df_valid.merge(a, on='id')

## 지상 층 결측치 채움

In [72]:
# 지상 층수는 상황에 나누어 채움
# 지워진 층수 데이터 합치고 채우기
a = df_te[['id', 'ttl_grnd_flr']]
df_test = df_test.merge(a, on='id')
df_test['ttl_grnd_flr'] = df_test.apply(lambda row:
                                        # 0이지만 0이 아니어야 할 때,
                                        round(row['ttl_ar'] / row['bldng_ar']) - row['ttl_dwn_flr']
                                        if (row['ttl_ar'] != 0) & (row['bldng_ar'] != 0) & (row['ttl_grnd_flr'] == 0) & (row['ttl_dwn_flr'] == 0)
                                        # null이 아니라면 있는 그대로
                                        else row['ttl_grnd_flr'] if pd.notnull(row['ttl_grnd_flr'])
                                        # 연면적 /빌딩면적 - 지하층수 
                                        else round(row['ttl_ar'] / row['bldng_ar']) - row['ttl_dwn_flr'] if ((row['ttl_ar'] != 0) & (row['bldng_ar'] != 0))
                                        # ttl_ar나 bldng_ar가 0이라면 0, axis=1)
                                        else 0, axis=1)


a = df_tr[['id','ttl_grnd_flr']]
df_train = df_train.merge(a, on='id')
df_train['ttl_grnd_flr'] = df_train.apply(lambda row:
                                        # 0이지만 0이 아니어야 할 때,
                                        round(row['ttl_ar'] / row['bldng_ar']) - row['ttl_dwn_flr']
                                        if (row['ttl_ar'] != 0) & (row['bldng_ar'] != 0) & (row['ttl_grnd_flr'] == 0) & (row['ttl_dwn_flr'] == 0)
                                        # null이 아니라면 있는 그대로
                                        else row['ttl_grnd_flr'] 
                                          if pd.notnull(row['ttl_grnd_flr'])
                                        # 연면적 /빌딩면적 - 지하층수 
                                        else round(row['ttl_ar'] / row['bldng_ar']) - row['ttl_dwn_flr']
                                          if ((row['ttl_ar'] != 0) & (row['bldng_ar'] != 0))
                                        # ttl_ar나 bldng_ar가 0이라면 0, axis=1)
                                        else 0, axis=1)

a = df_va[['id','ttl_grnd_flr']]
df_valid = df_valid.merge(a, on='id')
df_valid['ttl_grnd_flr'] = df_valid.apply(lambda row:
                                        # 0이지만 0이 아니어야 할 때,
                                        round(row['ttl_ar'] / row['bldng_ar']) - row['ttl_dwn_flr']
                                        if (row['ttl_ar'] != 0) & (row['bldng_ar'] != 0) & (row['ttl_grnd_flr'] == 0) & (row['ttl_dwn_flr'] == 0)
                                        # null이 아니라면 있는 그대로
                                        else row['ttl_grnd_flr'] if pd.notnull(row['ttl_grnd_flr'])
                                        # 연면적 /빌딩면적 - 지하층수 
                                        else round(row['ttl_ar'] / row['bldng_ar']) - row['ttl_dwn_flr'] if ((row['ttl_ar'] != 0) & (row['bldng_ar'] != 0))
                                        # ttl_ar나 bldng_ar가 0이라면 0, axis=1)
                                        else 0, axis=1)

In [73]:
# df_train[(df_train['lnd_ar'] != 0) & (df_train['lnd_ar'] < df_train['bldng_ar'])][w]

In [74]:
#df_train[(df_train['lnd_us_sttn_nm(clean)'] == '아파트') & (df_train['lnd_ar'] == 0)][w]

In [75]:
# 결측치가 얼마나 남았는지 체크하는 함수

def print_null(df):
#     print('건물채수가 0인 데이터: ', len(df[df['bldng_cnt'] == 0]), '개')
    print('건축면적이 0인 데이터: ', len(df[df['bldng_ar'] == 0]), '개')
    print('건물연면적 0인 데이터: ', len(df[df['ttl_ar'] == 0]), '개')
    print('토지면적이 0인 데이터: ', len(df[df['lnd_ar'] == 0]), '개')        
    print('지상 층수 데이터 없는 데이터: ', len(df[df['ttl_grnd_flr'].isnull()]), '개')
    print('지하 층수 데이터 없는 데이터: ', len(df[df['ttl_dwn_flr'].isnull()]), '개')
    print('지상 층수가 0 : ', len(df[df['ttl_grnd_flr'] == 0]), '개')
    print('지하 층수가 0: ', len(df[df['ttl_dwn_flr'] == 0.0]), '개')
    
print_null(df_train)

건축면적이 0인 데이터:  9659 개
건물연면적 0인 데이터:  8425 개
토지면적이 0인 데이터:  25567 개
지상 층수 데이터 없는 데이터:  0 개
지하 층수 데이터 없는 데이터:  0 개
지상 층수가 0 :  7696 개
지하 층수가 0:  55576 개


In [27]:
# df_test.to_csv('test_PCA_jmk_lnd_fl.csv', index=False)
# df_train.to_csv('train_PCA_jmk_lnd_fl.csv', index=False)
# df_valid.to_csv('valid_PCA_jmk_lnd_fl.csv', index=False)

## 빌딩면적, 연면적 결측치

- 토지면적이 모두 있는 데이터 안에서의 상관계수 보기

In [28]:
D = df_train
A = D['ttl_ar'] == 0
B = D['bldng_ar'] == 0
C = D['lnd_ar'] == 0
us_cnt = pd.DataFrame()


d = D[A & B & C]
us_cnt['nnn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & B & ~C] 
us_cnt['nny'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & ~B & ~C] 
us_cnt['nyy'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & ~B & C]
us_cnt['nyn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & B & C] 
us_cnt['ynn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & ~B & C] 
us_cnt['yyn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & B & ~C] 
us_cnt['yny'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & ~B & ~C]
us_cnt['yyy'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

us_cnt['sum'] = us_cnt.apply(lambda row : row.sum(), axis = 1)
us_cnt.loc['sum'] = us_cnt.apply(lambda col : col.sum())

In [29]:
df_tr_nn = df_train[~A & ~B & ~C]

In [30]:
df_tr_nn[w].corr().iloc[2:,2:]

,bldng_ar,ttl_ar,lnd_ar,ttl_grnd_flr,ttl_dwn_flr
bldng_ar,1.000000,0.912741,0.769379,0.553286,0.686816
ttl_ar,0.912741,1.000000,0.881332,0.666441,0.802816
lnd_ar,0.769379,0.881332,1.000000,0.439183,0.663700
ttl_grnd_flr,0.553286,0.666441,0.439183,1.000000,0.735388
ttl_dwn_flr,0.686816,0.802816,0.663700,0.735388,1.000000


In [31]:
# res = ols('lnd_ar ~ bldng_ar + ttl_ar', data = df_tr_nn).fit()
# res.summary()

---
<br>

### 빌딩면적 결측값 채우기 (bldng_ar)
#### 1. 연면적이 있고 층수가 1일 이상일 때, 
####       bldng_ar = around(bldng_ar = ttl_ar / 총 층수)

In [32]:
D = df_train
A = D['ttl_ar'] == 0
B = D['bldng_ar'] == 0
C = D['lnd_ar'] == 0
D[C].count()[0]

25567

In [38]:
# for df in df_l :

df = df_train

df['bldng_ar'] = df.apply(lambda row :
                          row['bldng_ar'] if row['bldng_ar'] != 0
                          else row['ttl_ar'] / (row['ttl_grnd_flr'] + row['ttl_dwn_flr']) if (row['ttl_grnd_flr'] + row['ttl_dwn_flr'] >= 1) & (row['ttl_ar']!= 0)
                          else row['ttl_ar'] if (row['ttl_ar'] != 0)
                          else row['bldng_ar'] , axis = 1)

df = df_test

df['bldng_ar'] = df.apply(lambda row :
                          row['bldng_ar'] if row['bldng_ar'] != 0
                          else row['ttl_ar'] / (row['ttl_grnd_flr'] + row['ttl_dwn_flr']) if (row['ttl_grnd_flr'] + row['ttl_dwn_flr'] >= 1) & (row['ttl_ar']!= 0)
                          else row['ttl_ar'] if (row['ttl_ar'] != 0)
                          else row['bldng_ar'] , axis = 1)

df = df_valid

df['bldng_ar'] = df.apply(lambda row :
                          row['bldng_ar'] if row['bldng_ar'] != 0
                          else row['ttl_ar'] / (row['ttl_grnd_flr'] + row['ttl_dwn_flr']) if (row['ttl_grnd_flr'] + row['ttl_dwn_flr'] >= 1) & (row['ttl_ar']!= 0)
                          else row['ttl_ar'] if (row['ttl_ar'] != 0)
                          else row['bldng_ar'] , axis = 1)

#### bldng_ar = around(lnd_ar * 상관계수)

DF blm_train

blm = 빌딩면적 / 토지면적 의 평균

tlm = 연면적 / 토지면적의 평균


## 토지면적 아웃라이어 수정 (아파트, 목장 등에서 많이 발견)

In [39]:
# # 아웃라이어 삭제
# for df in df_l:
#     df['lnd_ar'] = df.apply(lambda row :
#                         0 if row['lnd_ar'] == 0
#                         else row['lnd_ar'] if (row['lnd_ar'] >= row['ttl_ar']) | (row['lnd_ar'] >= row['bldng_ar'])
#                         else (row['bldng_ar'] * 3) if (row['bldng_ar'] != 0) & (row['lnd_ar'] <= row['bldng_ar'])
#                         else row['lnd_ar']
#                         ,axis = 1)    


In [44]:
# lnd_ar과 상관관계가 높은 변수를 고르는 함수 (building or total)
def find_bort(a, use):
    if len(a) != 0 :
        co = a[w].corr()
        t = co.loc['lnd_ar', 'ttl_ar']
        b = co.loc['lnd_ar', 'bldng_ar']
        bort = 't' if t >= b else 'b'
        return bort
    else :
        return 't'


# 토지 이용도 마다 최적의 면적대비율을 찾아 표로 만들기
use_l = df_train['lnd_us_sttn_nm(clean)'].unique()

    #train으로만 학습할 것임
blm_train = pd.DataFrame()
df = df_train

for use in use_l :
    a = df[~A & ~B & ~C & (df['lnd_us_sttn_nm(clean)'] == use)]
    
    bl = a['bldng_ar'] / a['lnd_ar']
    tl = a['ttl_ar'] / a['lnd_ar']
    
    blm = round(bl.mean(), 2)
    tlm = round(tl.mean(), 2)

    bort = find_bort(a, use)
#     print('use: ', use)
#     print('bl: ', bl)
#     print('tl: ', tl)
#     print('blm: ', blm)
#     print('tlm: ', tlm)
    s = {'bort': bort, 'tlm': tlm, "blm" : blm, 'lnd_us_sttn_nm(clean)' : use}
    blm_train = blm_train.append(s, ignore_index =True)
    
blm_train = blm_train.fillna(0)

blm_medi = blm_train['blm'].median()
tlm_medi = blm_train['tlm'].median()
print(blm_medi, tlm_medi)


# blm_mean = blm_train['blm'].mean()
# tlm_mean = blm_train['tlm'].mean()
# print(blm_mean, tlm_mean)

0.27 0.29000000000000004


In [45]:
# 도시주거 = []
# 도시상업 = []
# 도시공업 = []
# 도시녹지 = []
# 관리계획 
# 관리생산
# 관리보전
# 농림
# 자연환경보전지역



# 단독 - 제1종 전용주거지역
# 상업용 - 
# 아파트 - 3종 일반주거지역

In [46]:
blm_train

,blm,bort,lnd_us_sttn_nm(clean),tlm
0,0.32,b,단독,0.43
1,0.39,b,업무용,1.13
2,0.58,b,상업용,1.58
3,1.68,t,아파트,6.70
4,0.51,b,주상용,1.16
5,0.45,t,주거기타,0.60
6,0.35,t,주거나지,0.64
7,0.45,b,주상기타,1.00
8,0.11,b,운동장등,0.16
9,0.26,t,도로등,0.32


#### 적용

In [52]:
joined_train = df_train.merge(blm_train, how = 'left', on='lnd_us_sttn_nm(clean)')
joined_train[['bldng_ar', 'lnd_ar', 'lnd_us_sttn_nm(clean)', 'blm']]
df_train['bldng_ar'] = joined_train.apply(lambda row : row['bldng_ar'] if row['bldng_ar'] != 0
                                               else round(row['lnd_ar'] * row['blm']) if (row['lnd_ar'] != 0) & (row['blm'] != 0)
                                               else 0, axis=1)
# a['bldng_ar'] = a.apply(lambda row :
#                         round(row['lnd_ar'] / blm_train['blm']) if row['lnd_us_sttn_nm(clean)'] == 

## 건축면적 결측치

In [61]:
print_null(df_train)

건축면적이 0인 데이터:  8062 개
건물연면적 0인 데이터:  8425 개
토지면적이 0인 데이터:  8062 개
지상 층수 데이터 없는 데이터:  0 개
지하 층수 데이터 없는 데이터:  0 개
지상 층수가 0 :  7696 개
지하 층수가 0:  55576 개


#### 특수 기타, 발전소는 데이터 사례가 없어 눌값을 전체 평균으로 뽑는다

In [54]:
# 데이터 프레임의 lnd_ar가 채워졌는지 확인하는 메서드

def check_yyn(df, use):
    return df[(df['lnd_us_sttn_nm(clean)'] == use) & (df['ttl_ar'] != 0) & (df['bldng_ar'] != 0) & (df['lnd_ar'] == 0)][w]

In [55]:
def see_bl(df, use):
    bort = find_bort(df)
    
    if bort == 'b' : 
        a = df[(df['lnd_us_sttn_nm(clean)'] == use)]
        bl = a['bldng_ar'] / a['lnd_ar']
        blm = bl.mean()
        print(bl.mean())
        print(bl.median())
        sns.distplot(bl)
        return blm
    if bort == 't' :
        a = df[(df['lnd_us_sttn_nm(clean)'] == use)]
        bl = a['ttl_ar'] / a['lnd_ar']
        blm = bl.mean()
        print(bl.mean())
        print(bl.median())
        sns.distplot(bl)
        return blm

# 실행하기
def find_bl(df):
    a, bort = find_bort(df)
    
    if bort == 'b' : 
        bl = a['bldng_ar'] / a['lnd_ar']
        blm = bl.mean()
        return blm
    if bort == 't' :
        bl = a['ttl_ar'] / a['lnd_ar']
        blm = bl.mean()
        return blm

def fill_lnd(df, row, use, blm):
    
    if row['lnd_ar'] != 0:
        return row['lnd_ar']
    
    elif blm == 0 :
        return df['lnd_ar'].median()
    
    # ttl_ar == 0
    elif (row['ttl_ar'] == 0) :
        if row['bldng_ar'] == 0 :
            return 0
        elif row['bldng_ar'] != 0 :
            blm = find_bl(use, 'b')
            return row['bldng_ar'] / blm

    # ttl_ar != 0
    elif row['bldng_ar'] == 0 :
        return row['ttl_ar'] / blm

    # ttl_ar != 0 b_ar != 0
    else :
        return row['bldng_ar'] / blm                                   

In [56]:
joined = df_train.merge(blm_train, how = 'left', on='lnd_us_sttn_nm(clean)')
len(joined)

59191

In [60]:
#데이터프레임마다 토지사용목적에 따라
df_l = [df_train, df_test, df_valid]

for df in df_l :
    joined = df.merge(blm_train, how = 'left', on='lnd_us_sttn_nm(clean)')
    use_l = df['lnd_us_sttn_nm(clean)'].unique()
    
    df['lnd_ar'] = joined.apply(lambda row: 
                                row['lnd_ar'] if row['lnd_ar'] != 0
                                else 0 if (row['ttl_ar'] == 0) & (row['bldng_ar'] == 0)
                                else row['bldng_ar'] / row['blm'] if (row['bldng_ar'] != 0) & (row['bort'] == 'b') & (row['blm'] != 0) 
                                else row['ttl_ar'] / row['tlm'] if (row['ttl_ar'] != 0) &(row['bort'] == 't') & (row['tlm'] != 0)
                                else row['ttl_ar'] / tlm_medi if (row['ttl_ar'] != 0)
                                else row['bldng_ar'] / blm_medi if (row['bldng_ar'] != 0)
                                else 0 
                                , axis = 1)


## ttl_ar

In [64]:
for df in df_l:
    df['ttl_ar'] = df.apply(lambda row :
                            row['ttl_ar'] if row['ttl_ar'] != 0
                            else (row['bldng_ar'] * (row['ttl_grnd_flr'] + row['ttl_dwn_flr']) )if (row['bldng_ar'] != 0)* ((row['ttl_grnd_flr'] + row['ttl_dwn_flr']) >= 1)
                            else row['bldng_ar'] if (row['bldng_ar'] != 0)
                            else 0
                           ,axis = 1)

In [65]:
D = df_train
A = D['ttl_ar'] == 0
B = D['bldng_ar'] == 0
C = D['lnd_ar'] == 0
us_cnt = pd.DataFrame()


d = D[A & B & C]
us_cnt['nnn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & B & ~C] 
us_cnt['nny'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & ~B & ~C] 
us_cnt['nyy'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & ~B & C]
us_cnt['nyn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & B & C] 
us_cnt['ynn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & ~B & C] 
us_cnt['yyn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & B & ~C] 
us_cnt['yny'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & ~B & ~C]
us_cnt['yyy'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

us_cnt['sum'] = us_cnt.apply(lambda row : row.sum(), axis = 1)
us_cnt.loc['sum'] = us_cnt.apply(lambda col : col.sum())

us_cnt.reset_index()

,lnd_us_sttn_nm(clean),nnn,nny,nyy,nyn,ynn,yyn,yny,yyy,sum
0,골프장 회원제,1.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,8.0
1,공업기타,16.0,NaN,NaN,NaN,NaN,NaN,NaN,131.0,147.0
2,공업나지,13.0,NaN,NaN,NaN,NaN,NaN,NaN,102.0,115.0
3,공업용,71.0,NaN,NaN,NaN,NaN,NaN,NaN,2048.0,2119.0
4,공원등,19.0,NaN,NaN,NaN,NaN,NaN,NaN,27.0,46.0
5,과수원,164.0,NaN,NaN,NaN,NaN,NaN,NaN,121.0,285.0
6,기타,9.0,NaN,NaN,NaN,NaN,NaN,NaN,29.0,38.0
7,다세대,9.0,NaN,NaN,NaN,NaN,NaN,NaN,220.0,229.0
8,단독,3358.0,NaN,NaN,NaN,NaN,NaN,NaN,32039.0,35397.0
9,답,760.0,NaN,NaN,NaN,NaN,NaN,NaN,607.0,1367.0


In [ ]:
# #데이터프레임마다 토지사용목적에 따라
# for df in df_l :
#     use = ###########################################33
#     a = df[df['lnd_us_sttn_nm(clean)'] == use]
#     blm = find_bl(df, use)

#     a['lnd_ar'] = a.apply(lambda row: 
#                           row['lnd_ar'] if row['lnd_ar'] != 0
#                           else df['lnd_ar'].median() if blm == 0
#                           else 0 if (row['ttl_ar'] == 0) & (row['bldng_ar'] == 0)
#                           else row['bldng_ar'] / blm if (row['ttl_ar'] == 0) & (row['bldng_ar'] != 0 )
#                           else row['ttl_ar'] / blm if row['bldng_ar'] == 0
#                           else row['bldng_ar'] / blm 
#                           , axis = 1)
# #                               fill_lnd(a, row, use)
# #                               , axis = 1)
                              
        

In [ ]:
print_null(df_tr)
print_null(df_train)

### 토지임야, 조림, 여객자동차터미널 , '유해.혐오시설', 기타, 목장용지, 스키장, 공원묘지, 유원지, 특수기타, 위험시설, 콘도미니엄, 발전소

In [ ]:
see_bl(use, 't')

In [ ]:
df_train['lnd_ar'] = df_train.apply(lambda row : fill_lnd(row, use) if row['lnd_us_sttn_nm(clean)'] == use
                                     else row['lnd_ar']
                                    , axis =1)

In [ ]:
b['lnd_ar'] = b.apply(lambda row : row['bldng_ar'] / tlm)

In [ ]:
df_tr_nn['yong'] = df_tr_nn['bldng_ar'] / df_tr_nn['lnd_ar'] * 100

In [ ]:
import math

In [ ]:
df_tr['lnd_ar'].max()

In [ ]:
sns.lmplot(data=df_tr, x = 'lnd_ar', y ='bldng_ar' )

In [ ]:
# y = df_tr_nn['lnd_ar'].apply(lambda x : math.log(x))
# a['y'] = y
# a = a.sort_values('y')
# px.line(a, y = 'y')
# df_tr_nn['lnd_ar']
sns.lmplot(data = df_tr_nn, x = 'lnd_ar', y ='bldng_ar' )

In [ ]:
yong_med = (pd.DataFrame(df_tr_nn.groupby('lnd_us_sttn_nm(clean)')['yong'].median())).reset_index()
yong_mean = (pd.DataFrame(df_tr_nn.groupby('lnd_us_sttn_nm(clean)')['yong'].mean())).reset_index()

yong_med.rename(columns={'yong':'medi'}, inplace=True)
yong_mean.rename(columns={'yong':'mean'}, inplace=True)
yong= yong_med.merge(yong_mean)
yong.rename(columns={'lnd_us_sttn_nm(clean)': 'use'}, inplace=True)

In [ ]:
yong

In [ ]:
# use에 따라 coef 반환
def return_coef(use):
    a = df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == use]
    res = ols('lnd_ar ~ ttl_ar' , data=a).fit()
    if (res.rsquared_adj >= 0.5) & (res.conf_int().iloc[1:,0] * res.conf_int().iloc[1:,1] > 0):
        return (res.params['ttl_ar'], res.params['Intercept'])
    else : return (yong[yong['use'] == use]['medi'], 0)


In [ ]:
def return_lnd_ar(row):
    for i in yong['use'] :
        print(i)
        if i == row['lnd_us_sttn_nm(clean)']:
            print(row['lnd_us_sttn_nm(clean)'])
            v = return_coef(i)
            return (v[0] * row['ttl_ar']) + v[1]

In [ ]:
return_lnd_ar(df_train.iloc[1:2])

In [ ]:
df_train['lnd_ar'] = df_train.apply(lambda row :
                                    row['lnd_ar'] if row['lnd_ar'] != 0
                                    else return_lnd_ar[row] if row['ttl_ar'] != 0
                                    else row['bldng_ar']
                                    , axis = 1)

In [ ]:
v = (1, 2)
v[1]

In [ ]:
a = df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == '공업용']
res = ols('lnd_ar ~ ttl_ar' , data=a).fit()
res.summary()

In [ ]:
sns.distplot(res.resid)

In [ ]:
a = df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == '공원등']
print(a[['id','ttl_ar', 'lnd_ar', 'yong']])
print(len(a))
px.scatter(a, x = 'yong', y = 'lnd_ar', hover_name='id', color='id')

In [ ]:
(df_train['lnd_ar'] == 0).count()

In [ ]:
# 토지 면적 채우기

for i in yong['use']:
    
df_train['lnd_ar'] = df_train.apply(lambda row : 
                                   )


yong

In [ ]:
df_tr_nn['bld/lnd'] = df_tr_nn['bldng_ar'] / df_tr_nn['lnd_ar']

df_tr_nn[['bld/lnd', 'ttl_ar', 'bldng_ar']]

# 면적 데이터가 아에 없는 경우

In [ ]:
df_test[(df_test['ttl_ar'] == 0) & (df_test['lnd_ar'] == 0) & (df_test['bldng_ar']==0)][w]

In [ ]:
data = df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == '단독']

hover_data= ['jmk', 'address', 'bldng_ar','lnd_ar', 'bldng_cnt', 'ttl_ar', 'bldng_cnt_in_50m', 'cctv_dstnc', 'cctv_in_100m',
       'dayofweek', 'fire_yes', 'fr_mn_cnt', 'fr_sttn_dstnc',
       'fr_wthr_fclt_dstnc', 'fr_wthr_fclt_in_100m', 'hm_cnt', 'hmdt', 'year-month-day', 'hour', 'minute', 'mlt_us_yn(encode)',
       'no_tbc_zn_dstnc', 'sft_emrgnc_bll_dstnc', 'tag',
       'tbc_rtl_str_dstnc', 'tmprtr', 'wnd_drctn', 'wnd_spd', 'cluster', 'Comp1', 'Comp2','lnd_us_sttn_nm(clean)', 'ttl_dwn_flr', 'ttl_grnd_flr', 'bld/lnd', 'ttl/lnd']

px.scatter(data, x='ttl_ar'
           , y='lnd_ar', hover_name='id', hover_data= hover_data)

# plt.title("평균 바플롯")
# sns.barplot(x='bld/lnd', y='lnd_us_sttn_nm(clean)', data=data )
# plt.show()

In [ ]:
data = df_tr_nn[df_tr_nn['bld/lnd'] < 1]
plt.figure(figsize=(20,20))
sns.boxplot(x='bld/lnd', y='lnd_us_sttn_nm(clean)', data=data, whis=np.inf)
sns.stripplot(x='bld/lnd', y='lnd_us_sttn_nm(clean)', data=data, jitter=True, color="0.4")

# sns.catplot(data = data, x='bld/lnd', y='lnd_us_sttn_nm(clean)', kind='box', height=8, aspect=3)
plt.show()


### 01. bldng_ar 채우기

In [ ]:
def find_yong()

df_train['bldng_ar'] = df_train.apply(lambda row :
                                      row['bldng_ar'] if row['bldng_ar'] != 0
                                      else 
                                      
                                     , axis=1)

In [ ]:
fig = px.scatter(df_tr_nn, x = 'bldng_ar', y = 'lnd_ar', color = 'lnd_us_sttn_nm(clean)')

fig.update_layout(xaxis_rangeslider_visible=True)

In [ ]:
df_train[(df_train['lnd_us_sttn_nm(clean)'] == '공원묘지')]

In [ ]:
len(df_train[(df_train['lnd_us_sttn_nm(clean)'] == '공원묘지') & (df_train['fire_yes'] == 1)])

In [ ]:
px.scatter(df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == '스키장'], x = 'bldng_ar', y = 'lnd_ar')

In [ ]:
mean = df_train[(df_train['bldng_ar'] != 0) & (df_train['lnd_ar'] != 0)].groupby('lnd_us_sttn_nm(clean)').mean()[['bldng_ar', 'lnd_ar']]